This function returns routing dict: *paths*

In [1]:
import networkx as nx
from itertools import permutations,product
def generateClosPodsPaths(links, K, tapering_num = 0):
    """Generate the shortest path routing with balanced link loads for the clos-pods structure.
       The routing algorithm is compatiable with the algorithm described in http://ccr.sigcomm.org/online/files/p63-alfares.pdf
    Args:
        links (dict): Dictionary of the link information (key, value) = (linkID, (src, dst)).
        K (int): The number of pods in the clos-pods structure.
    Returns:
        list: The routing list Paths[src][dst] = [all transversed nodes].
    Examples:
        paths = generateClosPodsPaths(links, 4)
    """
    numCoreSwitches = K**2 / 4
    numHosts = K**3 / 4
    numSwitchesPerPod = K
    numHostsPerPod = K**2 / 4
    numSwitchPorts = K

    assert (numCoreSwitches.is_integer() and numHosts.is_integer()),"K is not appropriate!"
    numCoreSwitches = int(numCoreSwitches)
    numHosts = int(numHosts)
    numHostsPerPod = int(numHostsPerPod)

    G = nx.Graph()
    G.add_edges_from(links.values())
    paths = (nx.shortest_path(G))

    # Intra-Pod inter-EdgeSwitch routing
    # If source is the ith host in mth edge switch and destination is the jth host in nth edge switch,
    # the routing will transverse ath aggregation switch, a = (j + m) mod (k/2).
    # The remaining transversed nodes are then deterministic.
    for podID in range(K):
        for esrc, edst in permutations(range(1, numSwitchesPerPod // 2 + 1),2):
            for hsrc, hdst in product(range(1, numSwitchPorts // 2+1),range(1, numSwitchPorts // 2 + 1)):
                eOutPort = (hdst - 1 + esrc - 1) % (numSwitchesPerPod // 2) + 1
                hSrcID = podID * numHostsPerPod + (esrc - 1)*numSwitchPorts // 2 + hsrc
                hDstID = podID * numHostsPerPod + (edst - 1)*numSwitchPorts // 2 + hdst
                eSrcID = podID * numSwitchesPerPod + esrc
                eDstID = podID * numSwitchesPerPod + edst
                aID    = podID * numSwitchesPerPod + numSwitchesPerPod // 2 + eOutPort
                paths["h" + str(hSrcID)]["h" + str(hDstID)] = ['h' + str(hSrcID),'s' + str(eSrcID),'s' + str(aID),'s' + str(eDstID),'h' + str(hDstID)]
    # Inter-Pod routing
    # If source is the ith host in mth edge switch within pod k1, and destination is the jth host in nth edge switch within pod k2,
    # the routing will transverse ath aggregation switch in pod k1, a = (j + m) mod (k/2),
    # and transverse the kth core switch, k = (a + m) mod (k/2).
    # The remaining transversed nodes are then deterministic.

    for psrc, pdst in permutations(range(K),2):
        for esrc, edst in product(range(1, numSwitchesPerPod // 2 + 1), range(1, numSwitchesPerPod // 2 + 1)):
            for hsrc, hdst in product(range(1, numSwitchPorts // 2 + 1),range(1, numSwitchPorts // 2 + 1)):
                eOutPort = (hdst - 1 + esrc -1) % (numSwitchesPerPod // 2) + 1
                hSrcID = psrc * numHostsPerPod + (esrc - 1) * numSwitchPorts // 2 + hsrc
                hDstID = pdst * numHostsPerPod + (edst - 1)*numSwitchPorts // 2 + hdst
                eSrcID = psrc * numSwitchesPerPod + esrc
                eDstID = pdst * numSwitchesPerPod + edst
                aSrcID = psrc * numSwitchesPerPod + numSwitchesPerPod // 2 + eOutPort
                aDstID = pdst *numSwitchesPerPod + numSwitchesPerPod // 2 + eOutPort
                aOutPort = (hdst - 1 + eOutPort - 1) % (numSwitchesPerPod // 2 - tapering_num) + 1
                coreID = numSwitchesPerPod * K + (eOutPort - 1) * numSwitchPorts // 2 + aOutPort

                paths["h" + str(hSrcID)]["h" + str(hDstID)] = ['h' + str(hSrcID),'s' + str(eSrcID),'s' + str(aSrcID),'s' + str(coreID),'s' + str(aDstID),'s' + str(eDstID),'h' + str(hDstID)]        

    return paths

def ClosPods_get_topo(K=4, tapering_num=0):
    
    numCoreSwitches = K**2 / 4
    numHosts = K**3 / 4
    numSwitchesPerPod = K
    numHostsPerPod = K**2 / 4
    numSwitchPorts = K
    
    assert (numCoreSwitches.is_integer() and numHosts.is_integer()),"K is not appropriate!"
    numCoreSwitches = int(numCoreSwitches)
    numHosts = int(numHosts)
    numHostsPerPod = int(numHostsPerPod)
    
    links = {}
    hosts = ['h'+str(i) for i in range(1,int(K**3/4+1))]
    linkID = 1
    for pod in range(K):
        hostIDStart = pod * numHostsPerPod + 1
        coreSwitchStart = numSwitchesPerPod * K + 1
        EdgeSwitchStart =  pod*numSwitchesPerPod + 1
        EdgeSwitchEnd = int((pod + 1/2) * numSwitchesPerPod)
        aggSwitchStart = EdgeSwitchEnd + 1
        aggSwitchEnd = EdgeSwitchEnd + numSwitchesPerPod // 2

        for eS in range(EdgeSwitchStart,EdgeSwitchEnd + 1):
            for h in range(numSwitchPorts // 2):         
                links[linkID] = ('h'+str(hostIDStart + h), 's'+str(eS))
                linkID += 1
            hostIDStart += numSwitchPorts // 2
        for aS in range(aggSwitchStart,aggSwitchEnd + 1):
            for eS in range(EdgeSwitchStart,EdgeSwitchEnd + 1):
                links[linkID] = ('s'+str(aS), 's'+str(eS))
                linkID += 1
            for cS in range(numSwitchPorts//2):
                links[linkID] = ('s'+str(coreSwitchStart + cS), 's'+str(aS))
                linkID += 1
            coreSwitchStart += numSwitchPorts//2
    # # links are directed
    # for i in range(linkID - 1):
    #     links[i + linkID] = (links[i + 1][1], links[i + 1][0])
    switches = ['s'+str(i) for i in range(1, int(1+K**2 / 4+K**2))]
    
    if tapering_num>0:
        tmp=tapering_num/(K/2)
        
        assert (tmp.is_integer() and tapering_num!=numCoreSwitches),"tapering is not appropriate!"

        num_pods_switch =  K**2
        core_removed = []
        for idx in range(num_pods_switch+1,num_pods_switch+K**2 // 4+1):
            if (idx-num_pods_switch-1)%(K/2) >= K/2 - tmp:
                switches.remove('s'+str(idx))
                core_removed.append('s'+str(idx))
        links = {k: v for k, v in links.items() if v[0] not in core_removed and v[1] not in core_removed}
    nodes = switches+hosts
    return nodes, links

The following code generates topology (list: *links*) for given *K*.

C is capacity of links

In [2]:
# K = args.k
K=4
numCoreSwitches = K**2 / 4
numHosts = K**3 / 4
numSwitchesPerPod = K
numHostsPerPod = K**2 / 4
numSwitchPorts = K

assert (numCoreSwitches.is_integer() and numHosts.is_integer()),"K is not appropriate!"
numCoreSwitches = int(numCoreSwitches)
numHosts = int(numHosts)
numHostsPerPod = int(numHostsPerPod)

links = {}
hosts = ['h'+str(i) for i in range(1,K**3//4+1)]
linkID = 1
for pod in range(K):
    hostIDStart = pod * numHostsPerPod + 1
    coreSwitchStart = numSwitchesPerPod * K + 1
    EdgeSwitchStart =  pod*numSwitchesPerPod + 1
    EdgeSwitchEnd = int((pod + 1/2) * numSwitchesPerPod)
    aggSwitchStart = EdgeSwitchEnd + 1
    aggSwitchEnd = EdgeSwitchEnd + numSwitchesPerPod // 2

    for eS in range(EdgeSwitchStart,EdgeSwitchEnd + 1):
        for h in range(numSwitchPorts // 2):         
            links[linkID] = ('h'+str(hostIDStart + h), 's'+str(eS))
            linkID += 1
        hostIDStart += numSwitchPorts // 2
    for aS in range(aggSwitchStart,aggSwitchEnd + 1):
        for eS in range(EdgeSwitchStart,EdgeSwitchEnd + 1):
            links[linkID] = ('s'+str(aS), 's'+str(eS))
            linkID += 1
        for cS in range(numSwitchPorts//2):
            links[linkID] = ('s'+str(coreSwitchStart + cS), 's'+str(aS))
            linkID += 1
        coreSwitchStart += numSwitchPorts//2
# links are directed
for i in range(linkID - 1):
    links[i + linkID] = (links[i + 1][1], links[i + 1][0])

# C = {i:args.c for i in range(1, 2 * linkID - 1)}
C = {i:1 for i in range(1, 2 * linkID - 1)}

In [3]:
# tapering_num = 0  : no tapering
# tapering_num = 0.5: tapering half of spine links
paths = generateClosPodsPaths(links, K, tapering_num = 0) 
nodes,edges = ClosPods_get_topo(K=4)

In [4]:
paths['h1']['h12']

['h1', 's1', 's4', 's19', 's12', 's10', 'h12']

In [5]:
edges_idx = {item[1]:item[0] for item in edges.items()}
edges_idx

{('h1', 's1'): 1,
 ('h2', 's1'): 2,
 ('h3', 's2'): 3,
 ('h4', 's2'): 4,
 ('s3', 's1'): 5,
 ('s3', 's2'): 6,
 ('s17', 's3'): 7,
 ('s18', 's3'): 8,
 ('s4', 's1'): 9,
 ('s4', 's2'): 10,
 ('s19', 's4'): 11,
 ('s20', 's4'): 12,
 ('h5', 's5'): 13,
 ('h6', 's5'): 14,
 ('h7', 's6'): 15,
 ('h8', 's6'): 16,
 ('s7', 's5'): 17,
 ('s7', 's6'): 18,
 ('s17', 's7'): 19,
 ('s18', 's7'): 20,
 ('s8', 's5'): 21,
 ('s8', 's6'): 22,
 ('s19', 's8'): 23,
 ('s20', 's8'): 24,
 ('h9', 's9'): 25,
 ('h10', 's9'): 26,
 ('h11', 's10'): 27,
 ('h12', 's10'): 28,
 ('s11', 's9'): 29,
 ('s11', 's10'): 30,
 ('s17', 's11'): 31,
 ('s18', 's11'): 32,
 ('s12', 's9'): 33,
 ('s12', 's10'): 34,
 ('s19', 's12'): 35,
 ('s20', 's12'): 36,
 ('h13', 's13'): 37,
 ('h14', 's13'): 38,
 ('h15', 's14'): 39,
 ('h16', 's14'): 40,
 ('s15', 's13'): 41,
 ('s15', 's14'): 42,
 ('s17', 's15'): 43,
 ('s18', 's15'): 44,
 ('s16', 's13'): 45,
 ('s16', 's14'): 46,
 ('s19', 's16'): 47,
 ('s20', 's16'): 48}

In [6]:
path = paths['h1']['h12']
path_l = []
for x,y in zip(path, path[1:]):
    if (x,y) in edges_idx.keys():
        path_l.append("l"+str(edges_idx[(x,y)]))
    else:
        path_l.append("l"+str(edges_idx[(y,x)]))
path_l

['l1', 'l9', 'l11', 'l35', 'l34', 'l28']

In [7]:
g2_flows = []
cnt = 1
src_list = [node for node in paths.keys() if 'h' in node]
src_list.sort(key = lambda a : int(a[1:]))
for src in src_list:
    dst_list = [node for node in paths[src].keys() if 'h' in node]
    dst_list.sort(key = lambda a : int(a[1:]))
    for dst in dst_list:
        path = paths[src][dst]
        path_l = []
        for x,y in zip(path, path[1:]):
            if (x,y) in edges_idx.keys():
                path_l.append("l"+str(edges_idx[(x,y)]))
            else:
                path_l.append("l"+str(edges_idx[(y,x)]))

        g2_flows.append({'id': 'f'+str(cnt), 'path':path_l})
        cnt += 1

In [8]:
g2_flows

[{'id': 'f1', 'path': []},
 {'id': 'f2', 'path': ['l1', 'l2']},
 {'id': 'f3', 'path': ['l1', 'l5', 'l6', 'l3']},
 {'id': 'f4', 'path': ['l1', 'l9', 'l10', 'l4']},
 {'id': 'f5', 'path': ['l1', 'l5', 'l7', 'l19', 'l17', 'l13']},
 {'id': 'f6', 'path': ['l1', 'l9', 'l11', 'l23', 'l21', 'l14']},
 {'id': 'f7', 'path': ['l1', 'l5', 'l7', 'l19', 'l18', 'l15']},
 {'id': 'f8', 'path': ['l1', 'l9', 'l11', 'l23', 'l22', 'l16']},
 {'id': 'f9', 'path': ['l1', 'l5', 'l7', 'l31', 'l29', 'l25']},
 {'id': 'f10', 'path': ['l1', 'l9', 'l11', 'l35', 'l33', 'l26']},
 {'id': 'f11', 'path': ['l1', 'l5', 'l7', 'l31', 'l30', 'l27']},
 {'id': 'f12', 'path': ['l1', 'l9', 'l11', 'l35', 'l34', 'l28']},
 {'id': 'f13', 'path': ['l1', 'l5', 'l7', 'l43', 'l41', 'l37']},
 {'id': 'f14', 'path': ['l1', 'l9', 'l11', 'l47', 'l45', 'l38']},
 {'id': 'f15', 'path': ['l1', 'l5', 'l7', 'l43', 'l42', 'l39']},
 {'id': 'f16', 'path': ['l1', 'l9', 'l11', 'l47', 'l46', 'l40']},
 {'id': 'f17', 'path': ['l2', 'l1']},
 {'id': 'f18', 'pa